# Linking objects in panel

In the  [param user guide](Param.ipynb), we have seen how parameterized classes can be used to automatically generate a graphical user interface without any additional effort. However frequently it is necessary to manually set up links between widgets either through Python or JS based callbacks. Explicitly instantiating panel widgets affords more control over their attributes and therefore allows for more flexibility.

In this notebook we will focus on linking parameters of panel objects, specifically widgets to other objects. Specifically we will introduce two main APIs:

* ``link`` and ``jslink`` which are high-level APIs to perform linking either in Python or JS respectively
* ``obj.param.watch`` which is a lower level API provided by [param](https://param.pyviz.org) 

In [ ]:
import param
import numpy as np
import panel as pn
pn.extension()

## Linking objects in Python

As a very straightforward example let us start by creating a ``Markdown`` pane:

In [ ]:
markdown = pn.pane.Markdown('Markdown display')
markdown

Next let us create a ``TextInput`` widget:

In [ ]:
text_input = pn.widgets.TextInput(value=markdown.object)
text_input

We now have a number of options to link these two objects, either via a Python callback or within JS. So let's step through these options 1-by-1.

### The ``link`` method

The ``link`` method on Widgets provides a very straightforward means to link parameters on the widget to some other object. In the simplest case we simply provide the ``target`` object and define the mapping between the source and target parameters as the keywords, e.g. in this case we map the ``value`` parameter on the ``TextInput`` widget to the ``object`` parameter on the ``Markdown`` pane:

In [ ]:
text_input.link(markdown, value='object')

Having linked the value of the widget to the object parameter, updating the widget value will now update both:

In [ ]:
text_input.value = 'New text'

For more complex mappings between the widget value and , e.g. we can do some processing on the string before setting it on the Markdown object:

```python
def callback(target, event):
    target.object = event.new.title() + '!!!'

text_input.link(markdown, callbacks={'value': callback})
```

### The ``watch`` method

The ``link`` method provides a high-level API to link to parameters, which is adequate in most cases. However in some cases we want more control in which case we can fall back to the ``param.watch`` method, which is what panel uses internally to make all reactive features work. The main differences are that ``watch`` does not assume you are linking two objects, provides more control over what you are watching, allows batched callbacks when multiple parameters change at once and provides control over when an event is triggered, i.e. always or only when the parameter changes (the default).

To demonstrate this let us set up three different models, 1) a `Markdown` pane to display the possible options, 2) a ``Markdown`` pane to display the selected options and 3) a ``ToggleGroup`` widget which allows us to toggle between a number of options:

In [ ]:
selections = pn.pane.Markdown(object='Possible options: A, B, C, D')
selected = pn.pane.Markdown(object='Selected: None')
toggle = pn.widgets.ToggleGroup(options=['A', 'B', 'C'])

#### Defining a callback

Next we define a callback which can handle multiple parameter changes at once and uses the ``Event``'s ``name`` to figure out how to process the event, e.g. in this case it updates either the ``selections`` or the ``selected`` pane depending on whether ToggleGroup ``options`` or ``value`` changed:

In [ ]:
def callback(*events):
    print(events)
    for event in events:
        if event.name == 'options':
            selections.object = 'Possible options: %s' % ', '.join(event.new)
        elif event.name == 'value':
            selected.object = 'Selected: %s' % ','.join(event.new)

#### Event objects

Before going any further let us discover what these ``Event`` objects are. An ``Event`` is used to signal the change in a parameter value, they provide a number of useful attributes that provides additional information about the event:

* **``name``**: The name of the parameter that has changed
* **``new``**: The new value of the parameter
* **``old``**: The old value of the parameter before the event was triggered
* **``type``**: The type of event, can be one of 'triggered', 'changed' or 'set'
* **``what``**: Describes what about the parameter changed (usually the value but other parameter attributes can also change)
* **``obj``**: The Parameterized instance that holds the parameter
* **``cls``**: The Parameterized class that holds the parameter

#### Registering a watcher

Now that we know how to define a callback and use make use of ``Event`` attributes it is time to register the callback. The ``obj.param.watch`` method lets us supply the callback along with the parameters we want to watch. Additionally we can declare whether the events should only be triggered when the parameter value changes or whenever the parameter is set:

In [ ]:
watcher = toggle.param.watch(callback, ['options', 'value'], onlychanged=False)

Now let us display the widget alongside the ``Markdown`` panes which reflect the current state of the widget:

In [ ]:
pn.Row(pn.layout.WidgetBox(toggle, width=200), selections, pn.Spacer(width=50), selected)

If you toggle the widget values you will see the selected pane update in response. We can also trigger events manually either by setting a parameter as normal or by using the ``set_param`` method, which also allows us to batch the changes:

In [ ]:
toggle.param.set_param(options={'A': 'A', 'B': 'B', 'C': 'C', 'D': 'D'}, value=['D'])

As you can see by the ``print`` output the two parameter changes have been batched as a single call to the callback, which can be much more efficient.

#### Triggering events

If necessary it is also possible to trigger events manually without explicitly setting a parameter, which can be useful for refreshing some information even when the parameter itself hasn't change:

In [ ]:
toggle.param.trigger('options', 'value')

#### Unlinking

If for whatever reason we want to stop watching parameter changes we can unsubscribe using by passing our ``watcher`` which was returned in the ``watch`` call above to the ``unwatch`` method:

In [ ]:
toggle.param.unwatch(watcher)

## Linking objects in JS

Linking objects in Python is often very convenient, it allows writing code entirely in Python, however it also requires a live Python kernel. If instead we want a static example to have custom interactivity or we simply want to avoid the overhead of having to call back into Python we can also define links in Javascript.

### Linking panel components

To begin with let us see how we can express simple links between standard panel components before digging into some more specific and complex examples.

#### Linking model properties

Let us start by rehashing the example from above, linking the ``value`` of the ``TextInput`` widget to the ``object`` property of the ``Markdown`` pane:

In [ ]:
markdown = pn.pane.Markdown('Markdown display')
text_input = pn.widgets.TextInput(value=markdown.object)

link = text_input.jslink(markdown, value='object')

pn.Row(text_input, markdown)

As you can see the signature is identical as in the ``link`` example above, Panel translates the specification into a JS code snippet which syncs the properties on the underlying bokeh properties.

#### Linking using custom JS code

Since everything happens in JS we can't provide a Python callback, instead we can define a JS code snippet, which is executed when a property changes, e.g. we can define a  little code snippet which adds HTML bold tags (``<b>``) around the text before setting it on the target. The code argument should map from the parameter/propery on the source object to the code snippet to execute:

In [ ]:
markdown = pn.pane.Markdown('Markdown display')
text_input = pn.widgets.TextInput(value=markdown.object)

code = """
    target.text = '<b>' + source.value + '</b>'
"""
link = text_input.jslink(markdown, code={'value': code})

pn.Row(text_input, markdown)

Here ``source`` and ``target`` are made available in the Javascript namespace, allowing us to arbitrarily modify the models in response to property change events. Note however that the underlying bokeh model property names may differ slightly from the naming of the parameters on Panel objects, e.g. the 'object' parameter on the Markdown pane translates to the 'text' property on the bokeh model used to render the ``Markdown``.

### Linking Plots

Now that we have covered the basics let's see how we can use this approach to linking in some more complex examples involving different kinds of plots.

#### Linking Bokeh Plots

The ``jslink`` API trivially allows us to link a parameter on a panel widget to a bokeh plot property. Here we create a bokeh Figure with a simple sine curve. The ``jslink`` method allows us to pass any bokeh model held by the Figure as the ``target`` and then link the widget value to some property on it, e.g. here we link a ``FloatSlider`` value to the ``line_width`` of the ``Line`` glyph:

In [ ]:
from bokeh.plotting import figure

p = figure(width=300, height=300)
xs = np.linspace(0, 10)
r = p.line(xs, np.sin(xs))

width_slider = pn.widgets.FloatSlider(name='Line Width', start=0.1, end=10)
width_slider.jslink(r.glyph, value='line_width')

pn.Column(width_slider, p)

#### Linking HoloViews plots

Bokeh models allow us to directly access the underlying models and properties, when working with HoloViews on the other hand this link is a bit more indirect. Therefore HoloViews will make various models available directly in the namespace:

* **``cds``**: The bokeh ``ColumnDataSource`` model which holds the data used to render the plot
* **``glyph``**: The bokeh ``Glyph`` defining the style of the element
* **``glyph_renderer``**: The bokeh ``GlyphRenderer`` responsible for rendering the element
* **``plot``**: The bokeh ``Figure``
* **``xaxis``/``yaxis``**: The Axis models of the plot
* **``x_range``/``y_range``**: The x/y-axis ``Range1d`` models defining the axis ranges

All these are made available in the JS codes namespace if we decide to provide a JS code snippet but can also be referenced in the property mapping. We can map the widget value to a property on the ``glyph`` by providing a specification separated by periods, e.g. in this case we can map the value to the ``glyph.size``:

In [ ]:
import holoviews as hv
import holoviews.plotting.bokeh

size_widget = pn.widgets.FloatSlider(value=5, start=3, end=20, name='Size')
color_widget = pn.widgets.TextInput(value='black', name='Color')

points = hv.Points(np.random.rand(10, 2)).options(padding=0.1, line_color='black')

size_widget.jslink(points, value='glyph.size')
color_widget.jslink(points, value='glyph.fill_color')

pn.Row(points, pn.layout.WidgetBox(size_widget, color_widget))